In [24]:
%pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [25]:
import numpy as np
import pandas as pd

from imblearn.over_sampling import SMOTE

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score,train_test_split,GridSearchCV

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report,roc_auc_score

In [26]:
df = pd.read_csv('../stroke_data_cleaned.csv')
df.head()

,Unnamed: 0,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke,bmi_gr,health_status,age_gr
0,0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.600000,formerly smoked,1,Extremely Obese,hd_only,Old
1,1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,28.893237,never smoked,1,Overweight,No disease,Old
2,2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.500000,never smoked,1,Obese,hd_only,Old
3,3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.400000,smokes,1,Obese,No disease,Mature Adult
4,4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.000000,never smoked,1,Normal,ht only,Old


In [27]:
df.isnull().sum()

Unnamed: 0           0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
bmi_gr               0
health_status        0
age_gr               0
dtype: int64

In [28]:
df2 = df[df.columns[1:12]]
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             5110 non-null   object 
 1   age                5110 non-null   float64
 2   hypertension       5110 non-null   int64  
 3   heart_disease      5110 non-null   int64  
 4   ever_married       5110 non-null   object 
 5   work_type          5110 non-null   object 
 6   Residence_type     5110 non-null   object 
 7   avg_glucose_level  5110 non-null   float64
 8   bmi                5110 non-null   float64
 9   smoking_status     5110 non-null   object 
 10  stroke             5110 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 439.3+ KB


In [29]:
gender_labels = {'Male':0,'Female':1}
ever_married_labels = {'Yes':0,'No':1}
work_type_labels = {'Private' : 0, 'Self-employed' : 1, 'Govt_job' : 2, 'children' : 3, 'Never_worked' : 4}
residence_type_labels = {'Urban' : 0, 'Rural' : 1}
smoking_status_labels = {'formerly smoked' : 0, 'never smoked' : 1, 'smokes' : 2, 'Unknown' : 3}

df2['gender'] = df2['gender'].map(gender_labels)
df2['ever_married'] = df2['ever_married'].map(ever_married_labels)
df2['work_type'] = df2['work_type'].map(work_type_labels)
df2['Residence_type'] = df2['Residence_type'].map(residence_type_labels)
df2['smoking_status'] = df2['smoking_status'].map(smoking_status_labels)

df2.head()

C:\Users\harsh\AppData\Local\Temp/ipykernel_11720/243339935.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['gender'] = df2['gender'].map(gender_labels)
C:\Users\harsh\AppData\Local\Temp/ipykernel_11720/243339935.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['ever_married'] = df2['ever_married'].map(ever_married_labels)
C:\Users\harsh\AppData\Local\Temp/ipykernel_11720/243339935.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,0,67.0,0,1,0,0,0,228.69,36.600000,0,1
1,1,61.0,0,0,0,1,1,202.21,28.893237,1,1
2,0,80.0,0,1,0,0,1,105.92,32.500000,1,1
3,1,49.0,0,0,0,0,0,171.23,34.400000,2,1
4,1,79.0,1,0,0,1,1,174.12,24.000000,1,1


In [30]:
X = df2.drop(columns = ['stroke']).values
y = df2['stroke'].values

In [31]:
X

array([[  0.        ,  67.        ,   0.        , ..., 228.69      ,
         36.6       ,   0.        ],
       [  1.        ,  61.        ,   0.        , ..., 202.21      ,
         28.89323691,   1.        ],
       [  0.        ,  80.        ,   0.        , ..., 105.92      ,
         32.5       ,   1.        ],
       ...,
       [  1.        ,  35.        ,   0.        , ...,  82.99      ,
         30.6       ,   1.        ],
       [  0.        ,  51.        ,   0.        , ..., 166.29      ,
         25.6       ,   0.        ],
       [  1.        ,  44.        ,   0.        , ...,  85.28      ,
         26.2       ,   3.        ]])

In [32]:
y

array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

In [33]:
X = MinMaxScaler().fit_transform(X)
X

array([[0.        , 0.81689453, 0.        , ..., 0.80126489, 0.30126002,
        0.        ],
       [1.        , 0.74365234, 0.        , ..., 0.67902317, 0.21298095,
        0.33333333],
       [0.        , 0.97558594, 0.        , ..., 0.23451205, 0.25429553,
        0.33333333],
       ...,
       [1.        , 0.42626953, 0.        , ..., 0.12865848, 0.2325315 ,
        0.33333333],
       [0.        , 0.62158203, 0.        , ..., 0.51320284, 0.17525773,
        0.        ],
       [1.        , 0.53613281, 0.        , ..., 0.13922999, 0.18213058,
        1.        ]])

In [34]:
X_train,X_test,y_train,y_test = train_test_split(X,y.ravel(),test_size=0.3,random_state=33)

In [35]:
print("Before OverSampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train==0)))

sm = SMOTE(random_state=2)
x_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

print('After OverSampling, the shape of train_X: {}'.format(x_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))

Before OverSampling, counts of label '1': 168
Before OverSampling, counts of label '0': 3409 

After OverSampling, the shape of train_X: (6818, 10)
After OverSampling, the shape of train_y: (6818,) 

After OverSampling, counts of label '1': 3409
After OverSampling, counts of label '0': 3409


In [36]:
classifier = RandomForestClassifier(random_state= 34)
classifier.fit(x_train_res,y_train_res)

y_pred = classifier.predict(X_test)

In [37]:
accuracies = cross_val_score(
                estimator = classifier,
                X = x_train_res,
                y = y_train_res,
                scoring = 'accuracy',
                cv = 10
            )

print('classification report:')
print(classification_report(y_test,y_pred))
print('roc score:')
print(roc_auc_score(y_test,y_pred))
print('cross validation mean score:')
print(accuracies.mean()*100)

classification report:
              precision    recall  f1-score   support

           0       0.95      0.95      0.95      1452
           1       0.14      0.14      0.14        81

    accuracy                           0.91      1533
   macro avg       0.54      0.54      0.54      1533
weighted avg       0.91      0.91      0.91      1533

roc score:
0.5441409039893889
cross validation mean score:
96.14263998518652


In [38]:
grid_params = {
    'n_estimators':[100,150,200],
    'criterion':['gini','entropy']
    }

In [39]:
grid_search = GridSearchCV(
                estimator = classifier,
                param_grid = grid_params,
                scoring = 'accuracy',
                cv = 10,
            )

grid_search.fit(x_train_res,y_train_res)
y_pred = grid_search.predict(X_test)

In [40]:
print('best accuracy:',grid_search.best_score_)
print('best params:',grid_search.best_params_)

best accuracy: 0.9624540846865701
best params: {'criterion': 'entropy', 'n_estimators': 200}


In [41]:
grid_search2 = GridSearchCV(
                estimator = classifier,
                param_grid = grid_params,
                scoring = 'recall',
                cv = 10,
            )

grid_search2.fit(x_train_res,y_train_res)
y_pred = grid_search2.predict(X_test)

print('best accuracy:',grid_search2.best_score_)
print('best params:',grid_search2.best_params_)

best accuracy: 0.9727229601518026
best params: {'criterion': 'entropy', 'n_estimators': 150}


In [42]:
classifier = RandomForestClassifier(
                n_estimators = 200,
                criterion = 'entropy'
            )
classifier.fit(x_train_res,y_train_res)
y_pred = classifier.predict(X_test)

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.95      0.95      0.95      1452
           1       0.16      0.19      0.17        81

    accuracy                           0.91      1533
   macro avg       0.56      0.57      0.56      1533
weighted avg       0.91      0.91      0.91      1533



In [43]:
classifier2 = RandomForestClassifier(
                n_estimators = 150,
                criterion = 'entropy'
            )
classifier2.fit(x_train_res,y_train_res)
y_pred = classifier2.predict(X_test)

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.95      0.95      0.95      1452
           1       0.12      0.12      0.12        81

    accuracy                           0.91      1533
   macro avg       0.54      0.54      0.54      1533
weighted avg       0.91      0.91      0.91      1533

